## 표 편집

출처: https://school.programmers.co.kr/learn/courses/30/lessons/81303/

행 크기: n
시간복잡도: O(n)

#### 최초 아이디어
- 행을 배열로 만들자
- up & down은 배열 index를 이동시키자
- 삭제 연산은 각 index에 삭제 여부를 나타내는 True/False 값으로 표현하자
- 복구 연산은 삭제 연산 시 stack에 쌓아두고 LIFO로 꺼내쓰자
- 행의 크기가 크고 삭제 연산이 많아 배열 사이에 삭제된 index gap이 크면 up & down 오버헤드가 커질 수 있겠다
- 삭제되지 않은 각 index를 연결하는 배열 기반 링크드 리스트같은 걸 만들어보자

#### 아쉬웠던 점
- 최초 시도에서 링크드 리스트같은 걸 만들긴 했으나, 다소 가독성이 떨어짐. 배열에 prev, next를 갖는 Node를 넣어 관리했으면 더 깔끔했을듯.
- 어차피 배열 기반이라 링크드 리스트로 관리하지 않고, 각 index의 up & down 방향을 담은 index 배열을 사용할 수 있었는데 생각하지 못함. (2nd)

##### 생각하지 못한 Edge Case
- 배열 양 끝단 (0, n-1)의 up과 down 방향은 (-1, n)이 됨. 배열에 -1th index와 nth index로 접근하는 경우가 발생함.
  - index가 0보다 작고 n보다 크거나 같은 경우를 체크하여 배열을 변경시키지 않는 방법
  - 양 끝에 dummy 행 2개를 추가하고, 사용할 때는 1 ~ nth index의 배열의 상태만 확인하는 방법

In [ ]:
class LinkedList:
    def __init__(self, n, k):
        self.cursor = k
        self.arr = [[i, True, i-1, i+1] for i in range(n)]
        self.stack = []
        self.n = n

    def up(self, x):
        for i in range(x):
            self.cursor = self.arr[self.cursor][2]

    def down(self, x):
        for i in range(x):
            self.cursor = self.arr[self.cursor][3]

    def delete(self):
        curNode = self.arr[self.cursor]
        curNode[1] = False
        upIdx = curNode[2]
        downIdx = curNode[3]
        if upIdx >= 0:
            self.arr[upIdx][3] = downIdx
        if downIdx < self.n:
            self.arr[downIdx][2] = upIdx
        self.stack.append(self.cursor)
        if downIdx == self.n:
            self.cursor = upIdx
        else:
            self.cursor = downIdx

    def restore(self):
        lastDeletedIdx = self.stack.pop()
        self.arr[lastDeletedIdx][1] = True
        upIdx = self.arr[lastDeletedIdx][2]
        downIdx = self.arr[lastDeletedIdx][3]
        if upIdx >= 0:
            self.arr[upIdx][3] = lastDeletedIdx
        if downIdx < self.n:
            self.arr[downIdx][2] = lastDeletedIdx




def solution(n, k, cmd):
    ll = LinkedList(n, k)
    for c in cmd:
        if c[0] == 'U':
            x = int(c.split()[1])
            ll.up(x)
        elif c[0] == 'D':
            x = int(c.split()[1])
            ll.down(x)
        elif c[0] == 'C':
            ll.delete()
        else:
            ll.restore()
    return ''.join(map(lambda x: 'O' if x[1] else 'X', ll.arr))

In [ ]:


def solution(n, k, cmd):
    up = list(range(-1, n+1))
    down = list(range(1, n+3))
    stack = []
    k += 1
    for c in cmd:
        if c == 'C':
            down[up[k]] = down[k]
            up[down[k]] = up[k]
            stack.append(k)
            if down[k] > n:
                k = up[k]
            else:
                k = down[k]
        elif c == 'Z':
            last = stack.pop()
            down[up[last]] = last
            up[down[last]] = last
        else:
            direction, x = c.split()
            x = int(x)
            if direction == 'U':
                for i in range(x):
                    k = up[k]
            else:
                for i in range(x):
                    k = down[k]

    result = ['O' for _ in range(n)]
    for x in stack:
        result[x-1] = 'X'
    return ''.join(result)

